In [14]:
from obspy import Trace, UTCDateTime, Stream # type: ignore
from obspy.clients.fdsn import Client as FDSNclient # type: ignore
from obspy.core.inventory import Inventory # type: ignore
import matplotlib.pyplot as plt # type: ignore
from bs4 import BeautifulSoup # type: ignore
import requests # type: ignore
import logging.handlers
import numpy as np # type: ignore
import logging
from multiprocessing import Queue
import pandas as pd # type: ignore
from src.filtering import filtering_data
from src.fourier import fourier_transform
from src.core import main_processing
import concurrent.futures

In [ ]:
# source data
datasource = pd.read_csv("checkpoint1.csv", index_col="index")
datasource["datetime"] = datasource["datetime"].str.replace(" ","T")
datasource.head(2)

In [ ]:
datasource[
    (datasource["datetime"] > "2009-09-21")
    &(datasource["sensed"] == 1)
    #&(datasource["Int"] == "II")
    ]

In [ ]:
ds_filtered = datasource[
    (datasource["datetime"] > "2009-09-21")
    #&(datasource["sensed"] == 1)
    #&(datasource["dist_MORF"] < 300)
    ]

In [ ]:
l1 = ds_filtered.index

In [ ]:
error_table = []

# Define your processing function
def process_index(eq):
    try:
        d_t = datasource.iloc[eq]  # Get the row based on index eq
        
        url = 'rtserve.iris.washington.edu:18000'
        source = {
            "LX": {"MORF": ["BHE", "BHN", "BHZ"]}
        }

        loc = '--'
        earliest_date = "2009-09-21T15:18:14.919538Z"
        ref_time = UTCDateTime(str(d_t["datetime"]))
        client = FDSNclient("IRIS")
        gap = 120

        starttime = ref_time - gap  # 2 minutes before
        endtime = ref_time + gap  # 2 minutes after

        st = None
        
        # Attempt to print available streams
        for server, values in source.items():
            for channel, ch in values.items():
                for i in ch:
                    trace = client.get_waveforms(server, channel, loc, i, starttime, endtime, attach_response=True)
                    
                    if st is None:
                        st = trace
                    else:
                        st += trace  # Use += to concatenate traces

        st2 = st.copy()

        # Process components
        st_z = st.select(component="Z")
        st_n = st.select(component="N")
        st_e = st.select(component="E")

        filtered_z, time_vector_z = filtering_data(st2.select(component="Z")[0])
        filtered_n, time_vector_n = filtering_data(st2.select(component="N")[0])
        filtered_e, time_vector_e = filtering_data(st2.select(component="E")[0])

        # Process Z component
        if len(st_z) > 0:
            dom_freq_z, amp_z = fourier_transform(filtered_z, st_z[0])
            datasource.loc[eq, "Domin_freq_z"] = dom_freq_z
        else:
            print("No Z component traces available.")

        # Process N component
        if len(st_n) > 0:
            dom_freq_n, amp_n = fourier_transform(filtered_n, st_n[0])
            datasource.loc[eq, "Domin_freq_n"] = dom_freq_n
        else:
            print("No N component traces available.")

        # Process E component
        if len(st_e) > 0:
            dom_freq_e, amp_e = fourier_transform(filtered_e, st_e[0])
            datasource.loc[eq, "Domin_freq_e"] = dom_freq_e
        else:
            print("No E component traces available.")

        details = main_processing(filtered_z, gap, dom_freq_z)
        #display(details)

        for key, dicts in details.items():
            if key == 1:
                for stat, values in dicts.items():
                    if stat == "Vel Amp (m/s)":
                        datasource.loc[eq, "P-Vel amp (m/s)"] = values
                    elif stat == "Disp Amp (m)":
                        datasource.loc[eq, "P-Disp amp (m)"] = values
                    elif stat == "peak2peak":
                        datasource.loc[eq, "P-peak2peak"] = values
                    elif stat == "r":
                        datasource.loc[eq, "P-r"] = values
                    elif stat == "moment_history":
                        datasource.loc[eq, "P-moment_history"] = values
                    elif stat == "tau_c":
                        datasource.loc[eq, "P-tau_c"] = values    
                    else:
                        print(f"{key} error")
            elif key == 2:
                for stat, values in dicts.items():
                    if stat == "Vel Amp (m/s)":
                        datasource.loc[eq, "S-Vel amp (m/s)"] = values
                    elif stat == "Disp Amp (m)":
                        datasource.loc[eq, "S-Disp amp (m)"] = values
                    elif stat == "peak2peak":
                        datasource.loc[eq, "S-peak2peak"] = values
                    elif stat == "r":
                        datasource.loc[eq, "S-r"] = values
                    elif stat == "moment_history":
                        datasource.loc[eq, "S-moment_history"] = values
                    elif stat == "tau_c":
                        datasource.loc[eq, "S-tau_c"] = values    
                    else:
                        print(f"{key} error")
            elif key == "RMS":
                datasource.loc[eq, "RMS"] = dicts    
            elif key == "Energy":
                datasource.loc[eq, "Energy"] = dicts 
            elif key == "peak_freq":
                datasource.loc[eq, "peak_freq"] = dicts 
            elif key == "wavelength":
                datasource.loc[eq, "wavelength"] = dicts 
            elif key == "peak_disp":
                datasource.loc[eq, "peak_disp"] = dicts 
            elif key == "M0":
                datasource.loc[eq, "M0"] = dicts 
            else:
                datasource.loc[eq, "other infos"] = values

        print(f"...Finished processing {eq}")    
    except Exception as e:
        print(f"Error fetching index: {eq}, Error: {e}")
        error_table.append(eq)

from concurrent.futures import ThreadPoolExecutor
if __name__ == "__main__":
    with ThreadPoolExecutor(max_workers=12) as executor:
        executor.map(process_index, l1)

                       

In [ ]:
#datasource.to_csv("processed_ds2_partial.csv")